In [1]:
!pip install PyPDF2

You are using pip version 9.0.3, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import PyPDF2 as pypdf
import regex as re
import numpy as np
import csv
import pandas as pd

In [3]:
PDFfile = open("bmcc2019-2020.pdf", "rb")
pdfread = pypdf.PdfFileReader(PDFfile)
pdfread

In [8]:
y = pdfread.getPage(61)
y.extractText().replace('\n', '')

"                                                                  Bay Mills Community College  2019-2020  61 COURSE DESCRIPTIONS  AC  ACCOUNTING AC104    Payroll Accounting    3 CR This class is designed to teach students the skills, procedures, payroll. Basic principles and concepts of accounting and financial reporting will also be covered.   Co-requisites: MA101, and CS112 or CS121.  AC105 QuickBooks for Small Business    3 CR Upon successful completion of the course, students will know how to use a computerized bookkeeping software package for small businesses. Instruction includes information on creating accounts, writing checks, balancing accounts, navigating the program, and more. Prerequisite: None.   AC131 Accounting I    4 CR This is an introductory course in accounting, which includes the recording and reporting of business transactions, and the measuring, planning and controlling of business income, assets and equities.  Co-requisite: MA101 or permission of instructor.  AC

In [7]:
pdfread.getNumPages()

94

In [13]:
def get_data(csv_name, pdf_name, page_range, pattern, dep_group, abbrev_group, c_group, t_group,
            year, grad_status):
    """Takes in name of PDF in local folder and extracts data in given PAGE_RANGE 
    using regex expression PATTERN and given group numbers. YEAR is the catalog year.
    GRAD_STATUS is either None (if unknown), 'G', 'UG', or a number indicating at
    which number is the course graduate."""
    
    PDFfile = open(pdf_name, "rb")
    pdfread = pypdf.PdfFileReader(PDFfile)
    
    csv_writer = csv.writer(open(csv_name, 'w'))
    csv_writer.writerow(["Department Name", "Course Catalogue Number", "Course Name",
     "Course Description", "Graduate/Undergraduate", "Format", "Lab", 
     "Academic Catalogue Year"])
    FS_keys = ["agri", "food", "animal"]
    
    look_ahead = pattern + r'(.+?)(?=' + pattern + r')'
    
    dep_names = {}
    #test = page_range[0] #DEBUGGING
    for i in np.arange(page_range[0], page_range[1]):
        page_text = pdfread.getPage(i).extractText().replace('\n', '')
        matches = re.findall(look_ahead, page_text)
        last = re.sub(look_ahead, '', page_text)
        last = re.search(pattern + r'(.+)', last)
        #print(test) #DEBUGGING
        #test += 1 #DEBUGGING
        matches = matches + [(last.group(dep_group), last.group(abbrev_group), 
                    last.group(c_group), last.group(t_group), last.group(5))]
        
        for match in matches:  
            #DEBUG: print(match)
            # Getting full department name
            if not dep_names.get(match[abbrev_group - 1]):
                dep_names[match[abbrev_group - 1]] = match[dep_group - 1]
            dep_name = dep_names.get(match[abbrev_group - 1])
            
            # Other groups
            course_num = int(match[c_group - 1][:3])
            title = str(match[t_group - 1])
            desc = str(match[4]) # assuming description is group 5
            
            #Filtering
            for key in FS_keys:
                if (re.search(key, title, flags=re.IGNORECASE) or 
                    re.search(key, desc, flags=re.IGNORECASE)):
                    G_UG = grad_status
                    if grad_status:
                        if type(grad_status) == int:
                            G_UG = "UG"
                            if int(t.group(1)) >= grad_status:
                                G_UG = "G"
                    lab = False
                    if (re.search(r'Laboratory|Lab', title, flags=re.IGNORECASE) or 
                    re.search(r'Laboratory|Lab', desc, flags=re.IGNORECASE)):
                        lab = True
                    csv_writer.writerow([dep_name, course_num, title, desc, 
                                             G_UG, None, lab, year])
                                            # Leaving format blank for now
                        
    print(dep_names)
                
bmcc_pattern = r'([a-zA-Z]+)?\s+([\/a-zA-Z]+)([\d]+\w?)\s([^\d]+)\d\sCR\s'
#Partial Test set:
get_data('bmcc_sample.csv', "bmcc2019-2020.pdf", [61, 88], bmcc_pattern, 
         1, 2, 3, 4, '2019-2021', None)

{'AC': 'ACCOUNTING', 'AE': 'ENRICHMENT', 'AG': 'SCIENCE', 'BI': 'BIOLOGY', 'BU': 'BUSINESS', 'CH': 'CHEMISTRY', 'CJ': 'EMPHASIS', 'CIS': 'SYSTEMS', 'CS': '', 'CT': 'TECHNOLOGY', 'EC': 'EDUCATION', 'ED': 'EDUCATION', 'EN': 'ENGLISH', 'ES': '', 'GE': 'GEOGRAPHY', 'HL': '', 'HS': 'HISTORY', 'HU': 'HUMANITIES', 'JR': 'JOURNALISM', 'MT': 'TECHNOLOGY', 'MA': 'MATH', 'NA': 'STUDIES', 'NL': 'LANGUAGE', 'NP': 'PROGRAM', 'NS': 'SCIENCE', 'OE': 'EDUCATION', 'PE': 'EDUCATION', 'PS': 'SCIENCE', 'PY': 'PSYCHOLOGY', 'SI': 'SCIENCE', 'SO': 'SOCIOLOGY'}


In [16]:
pd.read_csv('bmcc_sample.csv')

,Department Name,Course Catalogue Number,Course Name,Course Description,Graduate/Undergraduate,Format,Lab,Academic Catalogue Year
0,ENRICHMENT,111,Reading Comprehension & Written Communication...,This course is designed to improve basic readi...,NaN,NaN,False,2019-2021
1,SCIENCE,110,Fundamentals of Sustainable Agriculture & Food...,This course will introduce students to the bas...,NaN,NaN,False,2019-2021
2,SCIENCE,110,Fundamentals of Sustainable Agriculture & Food...,This course will introduce students to the bas...,NaN,NaN,False,2019-2021
3,STUDIES,123,Native American Contributions to Society,"Throughout the course, students will look at t...",NaN,NaN,False,2019-2021
4,STUDIES,123,Native American Contributions to Society,"Throughout the course, students will look at t...",NaN,NaN,False,2019-2021
5,SCIENCE,101,Environmental Science (Lab required),ip to the environment. This course will provi...,NaN,NaN,True,2019-2021


In [104]:
sample = 'UNIVERSITY OF THE DISTRICT OF COLUMBIA  UNDERGRADUATE AND GRADUATE COURSE CATALOG 2019-2021  274 | Page  Studies textiles with an in-depth study of fiber classification, yarns, fabric construction, finishes and color applications.  Examines  the fabrics commonly used in fashion merchandise for sources of material, construction, comparative qualities and usage from the fashion viewpoint from fiber-to-product cycle. Explores  methods of fiber identification, yarns and construction.  CRIMINAL ADMINISTRATION and LAW ENFORCEMENT  FSEM 101C First Year Seminar (1) The First Year Seminar is a unique transition course designed to facilitate  integration into UDC-CCÕs learning community. The course enhances  skills for academic success, develops understanding of community college culture, provides individualized academic advising, and fosters  meaningful educational engagement. Also encourages active participation in the community college and to post-experience reflection.    IGED 110 Foundation Writing I (3) Focuses on expository writing. Includes selected readings and extensive practice in writing essays (e.g., analysis, comparison and contrast, cause and effect). Also reviews grammar and introduces the student to library resources.  IGED 120 Found. Quantitative Reasoning (3) Provides mathematical skills, knowledge, and understanding necessary to function in a technological society. Topics include problem solving, sets and logic, numeration and mathematical systems, linear equations and inequalities, and graphing. Lec. 3 hrs.  POLI 206C Introduction to American Government (3) Introduces the major principles of American government and politics. Focuses on major national institutions of the Presidency, Congress, and the courts.  Also examines federalism, civil rights and civil liberties, and political behavior and dynamics.  CRIM 100C Criminal Justice System (3) Provides an overview of the criminal justice system, including its main elements and functions Also examines the social, political, and cultural considerations that have influenced and shaped system functions and institutions.  CRIM 102C Criminology (3) Introduces  the study of crime using computer software applications. Examines different types of crime and the problems of crime analysis. Stresses the importance of a geographical and demographic analysis of the incidence of crime.  IGED 111 Foundation Writing II (3) Continues the study of the writing process begun in English Composition I. This course focuses on argumentation and analysis with extensive practice in writing and in- depth critical thinking through the use of supplemental readings.  Culminates in the writing of a research paper.  IGED 120 Found. Quantitative Reasoning (3) Continues 1535 101. Explores measurement and geometry, trigonometry of right triangles, consumer mathematics, and introduces probability and statistics. Lec. 3 hrs.  PSYC 137C Psychology of Adjustment (3) Emphasizes the understanding of everyday human behavior through the application of scientific principles derived from research. Examines foundations of psychological adjustment and the development of maladaptive behavior. Discusses human reactions evoked by stressful and frustrating environmental events, as well as reactions to internal sources of frustration. Examines procedures of psychological assessment, change, and newer methods of enhancing adjustment.  CRIM 271C Dynamics of Human Relations (3) Introduces students to theoretical analysis, current research findings, models of helping methods, intervention designs, and follow-up evaluations for the adult and juvenile in criminal justice settings. Provides opportunities to role play and assess r behavior, as well as interact with others -s in evaluation and feedback.  CRIM 203C Introduction to Forensic Investigations (3) '
rest = re.sub(udc_pattern + r'(.+?)(?=' + udc_pattern + r')', '',sample)
rest

'UNIVERSITY OF THE DISTRICT OF COLUMBIA  UNDERGRADUATE AND GRADUATE COURSE CATALOG 2019-2021  274 | Page  Studies textiles with an in-depth study of fiber classification, yarns, fabric construction, finishes and color applications.  Examines  the fabrics commonly used in fashion merchandise for sources of material, construction, comparative qualities and usage from the fashion viewpoint from fiber-to-product cycle. Explores  methods of fiber identification, yarns and construction.  CRIMINAL ADMINISTRATION and LAW ENFORCEMENT  CRIM 203C Introduction to Forensic Investigations (3) '

In [106]:
rest_match = re.search(udc_pattern + r'(.+)', rest)
print(rest_match.group(1))
print(rest_match.group(2))
print(rest_match.group(3))
print(rest_match.group(4))
print(rest_match.group(5))

AttributeError: 'NoneType' object has no attribute 'group'

In [15]:
sample = '20     COURSE  PREFIX  CODE,  TITLE AND COURSE DES CRIPTION   ACCOUNTING (ACC)   ACC 100 Fundamentals of Accounting (3)   Prerequisite:     This course is an introduction to accounting fundamentals. It includes journalizing, posting, worksheets,  financial reports, and data processing for accounting. One hour laboratory required.   ACC 200 Financial Accounting (3)      This course covers the theory and practice of accounting in services and merchandising firms. It  emphasizes the accounting cycle, periodic reporting, receivables, payables, inventories, and accounting  systems . One hour laboratory required.   ACC 201 Managerial Accounting (3)      This course is a continuation of ACC 200 with emphasis on manufacturing firms, and managerial  approaches for decision - making. One hour laborato ry required.   ACC 310 Government and Non - Profit   Accounting (3)   Prerequisite: Minimu  .   This course covers the theory and practice of accounting for governmental and not - for - profit units.  Topics include financial reporting; budgetary a ccounting; fund accounts; and nonprofit accounting for  entities such as public colleges, universities and health care providers .   ACC 350 Finance (3)    ACC 310,  BUS 265, ECO 200 or 201.   Introduction to the theories and applications associated with the functions of capital markets. This  includes the conceptual foundations of portfolio theory, risk management, and asset valuation. This  course will examine the markets for equity, money, and  bonds. One hour laboratory required.   AGRICULTURE (AGR)   AGR 103 Conservation Planning (4)   This course introduces students to the understanding natural resource conservation issues and concerns  to allow them to develop conservation plans necessary to improve   natural resource conditions applicable  to tribal lands.   AGR 105 Int roduction to Animal Science (4)   This course will introduce students to animal and livestock industries, production, systems and markets.  Students will not only learn about national product ion but include Navajo Nation livestock industry.   AGR 110 Monitoring/Assessments and Rangeland Management (4)   This course will introduce students to the various grazing management concept, identify local vegetation  communities, identification of local plan ts, as related to current Navajo producers grazing management  systems. Field trips and field works will serve as a lab for hands - on and field experience to learn range  and vegetation monitoring concepts.   '
dine_pattern = r'(\w+\s\(\w{2,3}\))?\s+([a-zA-Z]{2,3})\s([\d]+\w?)\s([^\d]+)\(\d\)\s+(.+?)'
last = re.sub(dine_pattern + '(?=' + dine_pattern + ')', '', sample)
l

'20     COURSE  PREFIX  CODE,  TITLE AND COURSE DES CRIPTION      AGR 110 Monitoring/Assessments and Rangeland Management (4)   This course will introduce students to the various grazing management concept, identify local vegetation  communities, identification of local plan ts, as related to current Navajo producers grazing management  systems. Field trips and field works will serve as a lab for hands - on and field experience to learn range  and vegetation monitoring concepts.   '

In [9]:
r'test' + r'test2'

'testtest2'

In [15]:
dine_pattern + r'(?=' + dine_pattern + r')'

'(\\w+\\s\\(\\w{2,3}\\))?\\s+([a-zA-Z]{2,3})\\s([\\d]+\\w?)\\s([^\\d]+)\\(\\d\\)\\s+(.+?)(?=(\\w+\\s\\(\\w{2,3}\\))?\\s+([a-zA-Z]{2,3})\\s([\\d]+\\w?)\\s([^\\d]+)\\(\\d\\)\\s+(.+?))'

In [33]:
sample
re.findall(dine_pattern + r'(.+)(?=' + dine_pattern + r')', sample)[0]

('ACCOUNTING (ACC)',
 'ACC',
 '100',
 'Fundamentals of Accounting ',
 'Prerequisite:     This course is an introduction to accounting fundamentals. It includes journalizing, posting, worksheets,  financial reports, and data processing for accounting. One hour laboratory required.   ACC 200 Financial Accounting (3)      This course covers the theory and practice of accounting in services and merchandising firms. It  emphasizes the accounting cycle, periodic reporting, receivables, payables, inventories, and accounting  systems . One hour laboratory required.   ACC 201 Managerial Accounting (3)      This course is a continuation of ACC 200 with emphasis on manufacturing firms, and managerial  approaches for decision - making. One hour laborato ry required.   ACC 310 Government and Non - Profit   Accounting (3)   Prerequisite: Minimu  .   This course covers the theory and practice of accounting for governmental and not - for - profit units.  Topics include financial reporting; budgetary a

In [65]:
if not None:
    print(True)

True
